<a href="https://colab.research.google.com/github/KhairulBasharbd/Parallel_Processing/blob/master/Matrix_multiplication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-rj1x9vp_
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-rj1x9vp_
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0d2ab99cccbbc682722e708515fe9c4cfc50185a
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4716 sha256=f1ab71069a2fc5404ce53e55005d33c57078e61b9ce1d672845fc34db2508373
  Stored in directory: /tmp/pip-ephem-wheel-cache-hs_91y7p/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
#include <stdio.h>
#include <cuda.h>
#include <fstream>
#include <iostream>
#include <sstream>
#include <string>

using namespace std;

__global__ void matproduct(int *l, int *m, int *n, int row1, int col1, int row2, int col2)
{
    int x = blockIdx.x;
    int y = blockIdx.y;
    int z = blockIdx.z;

    int i;
    n[(col2 * row1 * z) + (col2 * y) + x] = 0;

    for (i = 0; i < col1; i++)
    {
        n[(col2 * row1 * z) + (col2 * y) + x] += l[(col1 * row1 * z) + (col1 * y) + i] * m[(col2 * row2 * z) + (col2 * i) + x];
    }
}

int main()
{
    int num_matrices; //1000
    int row1; //25
    int col1; //25
    int row2; //25
    int col2; //25

    ifstream file("input.txt");
    if (!file.is_open())
    {
        std::cerr << "Unable to open file." << std::endl;
        return 1;
    }

    string line;
    while (getline(file, line))
    {
        istringstream iss(line);
        int k, m, n, p;

        if (iss >> k >> m >> n >> p)
        {
            std::cout << "K: " << k << ", M: " << m << ", N: " << n << ", p: " << p << std::endl;
            num_matrices = k;
            row1 = m;
            col1 = n;
            row2 = n;
            col2 = p;
        }
        else
        {
            std::cerr << "Failed to k, m, n, p from the line." << std::endl;
        }
    }

    int a[row1 * col1 * num_matrices];
    int b[row2 * col2 * num_matrices];
    int c[row1 * col2 * num_matrices];

    int *d, *e, *f;
    int i, j;

    cudaEvent_t start, end;
    cudaEventCreate(&start);
    cudaEventCreate(&end);
    cudaEventRecord(start);

    // Initialize matrices a and b
    int element = 1;
    for (int k = 0; k < num_matrices; k++)
    {
        for (i = 0; i < row1; i++)
        {
            for (j = 0; j < col1; j++)
            {
                a[(col1 * row1 * k) + (col1 * i) + j] = element++;
            }
        }

        element = 2;
        for (i = 0; i < row2; i++)
        {
            for (j = 0; j < col2; j++)
            {
                b[(col2 * row2 * k) + (col2 * i) + j] = element++;
            }
        }
    }

    cudaMalloc((void **)&d, row1 * col1 * num_matrices * sizeof(int));
    cudaMalloc((void **)&e, row2 * col2 * num_matrices * sizeof(int));
    cudaMalloc((void **)&f, row1 * col2 * num_matrices * sizeof(int));

    cudaMemcpy(d, a, row1 * col1 * num_matrices * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(e, b, row2 * col2 * num_matrices * sizeof(int), cudaMemcpyHostToDevice);

    dim3 grid(col2, row1, num_matrices);

    matproduct<<<grid, 1>>>(d, e, f, row1, col1, row2, col2);

    cudaMemcpy(c, f, row1 * col2 * num_matrices * sizeof(int), cudaMemcpyDeviceToHost);

    printf("\nProduct of matrices:\n");
    for (int k = 0; k < 1; k++)
    {
        printf("Matrix %d:\n", k + 1);
        for (i = 0; i < row1; i++)
        {
            for (j = 0; j < col2; j++)
            {
                printf("%d\t", c[(col2 * row1 * k) + (col2 * i) + j]);
            }
            printf("\n");
        }
        printf("\n");
    }

    cudaEventRecord(end);
    cudaEventSynchronize(end);
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, end);
    printf("\nTime taken: %f\n ", milliseconds);

    cudaFree(d);
    cudaFree(e);
    cudaFree(f);

    return 0;
}


K: 10, M: 10, N: 10, p: 10

Product of matrices:
Matrix 1:
3410	3465	3520	3575	3630	3685	3740	3795	3850	3905	
8110	8265	8420	8575	8730	8885	9040	9195	9350	9505	
12810	13065	13320	13575	13830	14085	14340	14595	14850	15105	
17510	17865	18220	18575	18930	19285	19640	19995	20350	20705	
22210	22665	23120	23575	24030	24485	24940	25395	25850	26305	
26910	27465	28020	28575	29130	29685	30240	30795	31350	31905	
31610	32265	32920	33575	34230	34885	35540	36195	36850	37505	
36310	37065	37820	38575	39330	40085	40840	41595	42350	43105	
41010	41865	42720	43575	44430	45285	46140	46995	47850	48705	
45710	46665	47620	48575	49530	50485	51440	52395	53350	54305	


Time taken: 117.480286
 
